In [1]:
import pandas as pd
import re

from sentence_transformers import SentenceTransformer
import numpy as np, faiss, pickle, pathlib

/Users/maximrimer/PycharmProjects/ai-operator/henv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
df_pairs = pd.read_parquet('../data/df_pairs_cln.parquet')

In [8]:
df_pairs.head()

,client_phone,dialogue_language,classification_category,classification_subcategory,client_text,operator_text
0,77766060358,kk,Кредит БЕЗ залога,ЕП - Статус погашения. Перепроверяет,"[PERSON], қайырлы [PERSON]. [PERSON] банк кеңе...","Иә, иә, [NUMBER], [NUMBER], [NUMBER] ы, [NUMBE..."
1,77766060358,kk,Кредит БЕЗ залога,ЕП - Статус погашения. Перепроверяет,[PERSON] клиенттің өзіне [PERSON] аласыз ба? М...,[NUMBER]. Қазір сізде қандай сұрақ болып тұр? ...
2,77766060358,kk,Кредит БЕЗ залога,ЕП - Статус погашения. Перепроверяет,"Иә, соны енді мен кеше төледім. Ақшам болмаған...","Мен сіз түсіндім, қазір неше [NUMBER] ақпаратт..."
3,77766060358,kk,Кредит БЕЗ залога,ЕП - Статус погашения. Перепроверяет,"[PERSON]. Махаббат, [PERSON], еркек [PERSON] а...","Кешіріңіз әлі желдеспе сізде. Иә, несие бойынш..."
4,77766060358,kk,Кредит БЕЗ залога,ЕП - Статус погашения. Перепроверяет,"[PERSON] де ше? [NUMBER], және де [NUMBER] тұр...","Біз сізде [NUMBER]. Ол қалып тұрған ақшаңыз, Я..."


In [9]:
df_pairs.shape

(24621, 6)

In [10]:
df_pairs = (df_pairs
            .dropna(subset=["client_text", "operator_text"])
            .drop_duplicates(subset=["client_text", "operator_text"]))

In [11]:
df_pairs.shape

(24606, 6)

In [5]:
MODEL_NAME = "intfloat/multilingual-e5-large"
model = SentenceTransformer(MODEL_NAME, device="mps")

In [12]:
passages = ["passage: " + q for q in df_pairs["client_text"].tolist()]
embs = model.encode(passages, batch_size=128, normalize_embeddings=True)

In [13]:
vec_dim = embs.shape[1]
print(vec_dim)

1024


In [14]:
index = faiss.IndexFlatIP(vec_dim)

In [15]:
index.add(np.asarray(embs, dtype="float32"))

In [19]:
def retrieve_operator_reply(query: str, k: int = 1) -> list[dict]:
    vec = model.encode(["query: " + query], normalize_embeddings=True)
    sims, idxs = index.search(np.asarray(vec, dtype="float32"), k)
    out = []
    for sim, idx in zip(sims[0], idxs[0]):
        if idx == -1:
            continue
        row = df_pairs.iloc[idx]
        out.append({
            "answer": row["operator_text"],
            "source": row["client_text"],
            "sim": float(sim)
        })
    return out

In [23]:
test_query = "В каком разделе мобильного приложения надо смотреть?"

In [26]:
print(retrieve_operator_reply(test_query)[0]["answer"])

Получается, у меня ограничение.


In [27]:
faiss.write_index(index, "../data/faiss_e5_transcripts.index")
df_pairs.to_parquet("../data/q_a_transcripts.parquet")